## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

#Enable Google Maps in Browser
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Ambovombe,-25.1667,46.0833,76.77,63,2,9.13,MG,2022-01-31 04:50:25,clear sky
1,1,Basco,20.4487,121.9702,70.59,72,99,27.58,PH,2022-01-31 04:50:26,overcast clouds
2,2,Carnarvon,-24.8667,113.6333,87.87,58,23,24.16,AU,2022-01-31 04:48:45,few clouds
3,3,Arman,59.7000,150.1667,7.59,78,100,16.89,RU,2022-01-31 04:50:26,overcast clouds
4,4,Mataura,-46.1927,168.8643,73.87,57,92,4.94,NZ,2022-01-31 04:46:42,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
2,2,Carnarvon,-24.8667,113.6333,87.87,58,23,24.16,AU,2022-01-31 04:48:45,few clouds
12,12,Geraldton,-28.7667,114.6000,84.61,48,0,21.85,AU,2022-01-31 04:50:28,clear sky
18,18,Vaini,-21.2000,-175.2000,81.45,70,100,15.82,TO,2022-01-31 04:45:54,overcast clouds
19,19,Butaritari,3.0707,172.7902,81.93,76,51,19.06,KI,2022-01-31 04:46:57,light rain
29,29,Flinders,-34.5833,150.8552,86.20,71,30,8.19,AU,2022-01-31 04:50:33,light rain
33,33,Kavaratti,10.5669,72.6420,81.54,73,1,9.10,IN,2022-01-31 04:50:34,clear sky
36,36,New Norfolk,-42.7826,147.0587,86.90,30,0,3.00,AU,2022-01-31 04:46:41,clear sky
37,37,Manavalakurichi,8.1333,77.3000,86.09,60,6,4.63,IN,2022-01-31 04:50:35,clear sky
41,41,Souillac,-20.5167,57.5167,85.35,89,75,12.66,MU,2022-01-31 04:50:37,light rain
43,43,Mahebourg,-20.4081,57.7000,85.24,89,75,12.66,MU,2022-01-31 04:50:37,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                96
City                   96
Lat                    96
Lng                    96
Max Temp               96
Humidity               96
Cloudiness             96
Wind Speed             96
Country                96
Date                   96
Current Description    96
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Carnarvon,AU,87.87,few clouds,-24.8667,113.6333,
12,Geraldton,AU,84.61,clear sky,-28.7667,114.6000,
18,Vaini,TO,81.45,overcast clouds,-21.2000,-175.2000,
19,Butaritari,KI,81.93,light rain,3.0707,172.7902,
29,Flinders,AU,86.20,light rain,-34.5833,150.8552,
33,Kavaratti,IN,81.54,clear sky,10.5669,72.6420,
36,New Norfolk,AU,86.90,clear sky,-42.7826,147.0587,
37,Manavalakurichi,IN,86.09,clear sky,8.1333,77.3000,
41,Souillac,MU,85.35,light rain,-20.5167,57.5167,
43,Mahebourg,MU,85.24,broken clouds,-20.4081,57.7000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
hotel_df.head(10)

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Carnarvon,AU,87.87,few clouds,-24.8667,113.6333,Hospitality Carnarvon
12,Geraldton,AU,84.61,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
18,Vaini,TO,81.45,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
19,Butaritari,KI,81.93,light rain,3.0707,172.7902,Isles Sunset Lodge
29,Flinders,AU,86.20,light rain,-34.5833,150.8552,Shellharbour Resort & Conference Centre
33,Kavaratti,IN,81.54,clear sky,10.5669,72.6420,Hotels in Lakshadweep Islands
36,New Norfolk,AU,86.90,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages
37,Manavalakurichi,IN,86.09,clear sky,8.1333,77.3000,Jebinrio home
41,Souillac,MU,85.35,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
43,Mahebourg,MU,85.24,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Carnarvon,AU,87.87,few clouds,-24.8667,113.6333,Hospitality Carnarvon
12,Geraldton,AU,84.61,clear sky,-28.7667,114.6000,Broadwater Mariner Resort
18,Vaini,TO,81.45,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
19,Butaritari,KI,81.93,light rain,3.0707,172.7902,Isles Sunset Lodge
29,Flinders,AU,86.20,light rain,-34.5833,150.8552,Shellharbour Resort & Conference Centre


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))